# Controllable generation via RL to let Elon Musk speak ill of DOGE
> How to control text generation through a sentiment classifier.



In [ ]:
# %pip install pfrl@git+https://github.com/voidful/pfrl.git
# %pip install textrl==0.2.15

In [ ]:
from textrl import TextRLEnv,TextRLActor
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer, AutoModelWithLMHead
import logging
import sys
import pfrl
import torch
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')

**Using a pre-trained model, it can generate elonmusk's style tweets.**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("huggingtweets/elonmusk")  
model = AutoModelWithLMHead.from_pretrained("huggingtweets/elonmusk")
model.eval()
model.cuda()

**a sentiment classifier for rl reward**

In [ ]:
sentiment = pipeline('sentiment-analysis',model="cardiffnlp/twitter-roberta-base-sentiment",tokenizer="cardiffnlp/twitter-roberta-base-sentiment",device=0,return_all_scores=True)

In [ ]:
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.CRITICAL)

In [ ]:
sentiment("dogecoin is bad")

In [ ]:
# demo how the tokenization works
tokens = tokenizer.tokenize("dogecoin is bad")
token_ids = tokenizer.convert_tokens_to_ids(tokens)

for tokens, token_id in zip(tokens, token_ids):
    print(f"{tokens} -> {token_id}")
    
ids = [329, 262, 3773, 50256]
tokens = tokenizer.convert_ids_to_tokens(ids)

for token_id, tokens in zip(ids, tokens):
    print(f"{token_id} -> {tokens}")

In [ ]:
# demo how the model works
output = model.generate(tokenizer.encode("dogecoin is bad", return_tensors='pt').cuda())
generated = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated)

In [ ]:
sentiment("dogecoin is bad")[0][0]['score']

set our text generation reward, inverse perplexity + sentiment classifier.
- inverse perplexity make sure the generated sentence probability will be high.
- sentiment classifier can make the generate more negative.

In [ ]:
class MyRLEnv(TextRLEnv):
    def get_reward(self, input_item, predicted_list, finish): # predicted will be the list of predicted token
      reward = 0
      if finish or len(predicted_list) >= self.env_max_length:
        predicted_text = tokenizer.convert_tokens_to_string(predicted_list[0])
        # sentiment classifier
        # print("input_item : ", input_item['input'])
        print("predicted_text : ", predicted_text)
        # print("predicted_list : ", predicted_list)
        # print(sentiment(input_item[0]+predicted_text))
        # reward = sentiment(input_item[0]+predicted_text)[0][0]['score'] * 10
        reward = sentiment(input_item['input']+predicted_text)[0][0]['score'] * 10
        print("reward : ", reward)
      return reward

**fit one example**

In [ ]:
observaton_list = [{'input':'i think dogecoin is'}]

In [ ]:
env = MyRLEnv(model, tokenizer, observation_input=observaton_list,compare_sample=1)
actor = TextRLActor(env,model,tokenizer)
agent = actor.agent_ppo(update_interval=100, minibatch_size=3, epochs=10)

In [ ]:
predicted_str = actor.predict(observaton_list[0])

In [ ]:
pfrl.experiments.train_agent_with_evaluation(
    agent,
    env,
    steps=300,
    eval_n_steps=None,
    eval_n_episodes=2,       
    train_max_episode_len=100,  
    eval_interval=10,
    outdir='elon_musk_dogecoin', 
)

loading the best result and predict.

In [ ]:
agent.load("./elon_musk_dogecoin/best")

In [ ]:
actor.predict(observaton_list[0])